In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
from os.path import join
import json
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import trange, tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
sys.path.append("/n/home12/binxuwang/Github/DiffusionLearningCurve")
sys.path.append("/Users/binxuwang/Github/DiffusionLearningCurve/")
from core.diffusion_nn_lib import UNetBlockStyleMLP_backbone
from core.toy_shape_dataset_lib import generate_random_star_shape_torch
from core.diffusion_basics_lib import *
from core.diffusion_edm_lib import *
from core.network_edm_lib import SongUNet, DhariwalUNet
from core.DiT_model_lib import *
from core.diffusion_nn_lib import UNetBlockStyleMLP_backbone
from circuit_toolkit.plot_utils import saveallforms, to_imgrid, show_imgrid
from pprint import pprint

saveroot = f"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiffusionSpectralLearningCurve"

In [4]:
from core.network_edm_lib import SongUNet, DhariwalUNet, create_unet_model
from core.diffusion_edm_lib import UNetBlockStyleMLP_backbone, EDMPrecondWrapper, EDMCNNPrecondWrapper, EDMDiTPrecondWrapper
import json
from easydict import EasyDict as edict
expname = "FFHQ32_UNet_CNN_EDM_1blocks_1x_wide128_fixednorm"
savedir = join(saveroot, expname)
# model_dict = torch.load(join(expdir, "model_final.pth"))
config = json.load(open(join(savedir, "config.json")))
unet = create_unet_model(edict(config))
unet.load_state_dict(torch.load(join(savedir, "model_final.pth")))
model_precd_CNN = EDMCNNPrecondWrapper(unet, sigma_data=0.5, sigma_min=0.002, sigma_max=80, rho=7.0)
model_precd_CNN = model_precd_CNN.to("cuda").eval();

total number of trainable parameters in the Score Model: 2570243
total number of parameters in the Score Model: 2570243


/tmp/ipykernel_666950/1899011077.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load(join(savedir, "model_final.pth")))


In [5]:
config


{'channels': 3,
 'img_size': 32,
 'layers_per_block': 1,
 'decoder_init_attn': True,
 'attn_resolutions': [0],
 'model_channels': 128,
 'channel_mult': [1],
 'dropout': 0.0,
 'label_dim': 0,
 'augment_dim': 0}

In [8]:
unet.dec

ModuleDict(
  (32x32_in0): UNetBlock(
    (norm0): GroupNorm()
    (conv0): Conv2d()
    (affine): Linear()
    (norm1): GroupNorm()
    (conv1): Conv2d()
    (norm2): GroupNorm()
    (qkv): Conv2d()
    (proj): Conv2d()
  )
  (32x32_in1): UNetBlock(
    (norm0): GroupNorm()
    (conv0): Conv2d()
    (affine): Linear()
    (norm1): GroupNorm()
    (conv1): Conv2d()
  )
  (32x32_block0): UNetBlock(
    (norm0): GroupNorm()
    (conv0): Conv2d()
    (affine): Linear()
    (norm1): GroupNorm()
    (conv1): Conv2d()
    (skip): Conv2d()
  )
  (32x32_block1): UNetBlock(
    (norm0): GroupNorm()
    (conv0): Conv2d()
    (affine): Linear()
    (norm1): GroupNorm()
    (conv1): Conv2d()
    (skip): Conv2d()
  )
  (32x32_aux_norm): GroupNorm()
  (32x32_aux_conv): Conv2d()
)

In [11]:
unet.enc["32x32_conv"].out_channels

128

In [ ]:
from core.network_edm_lib import UNetBlock, PositionalEmbedding, FourierEmbedding, Linear, Conv2d, GroupNorm, silu
class SongUNetResNet(torch.nn.Module):
    def __init__(self,
        img_resolution,                     # Image resolution at input/output.
        in_channels,                        # Number of color channels at input.
        out_channels,                       # Number of color channels at output.
        label_dim           = 0,            # Number of class labels, 0 = unconditional.
        augment_dim         = 0,            # Augmentation label dimensionality, 0 = no augmentation.

        model_channels      = 128,          # Base multiplier for the number of channels.
        # channel_mult        = 2,    # Per-resolution multipliers for the number of channels.
        channel_mult_emb    = 4,            # Multiplier for the dimensionality of the embedding vector.
        num_blocks          = 4,            # Number of residual blocks per resolution.
        attn_resolutions    = [16],         # List of resolutions with self-attention.
        dropout             = 0.10,         # Dropout probability of intermediate activations.
        label_dropout       = 0,            # Dropout probability of class labels for classifier-free guidance.
        # decoder_init_attn   = True,         # NEW ARGS, whether to add attn in the first decoder block
        embedding_type      = 'positional', # Timestep embedding type: 'positional' for DDPM++, 'fourier' for NCSN++.
        channel_mult_noise  = 1,            # Timestep embedding size: 1 for DDPM++, 2 for NCSN++.
        encoder_type        = 'standard',   # Encoder architecture: 'standard' for DDPM++, 'residual' for NCSN++.
        decoder_type        = 'standard',   # Decoder architecture: 'standard' for both DDPM++ and NCSN++.
        resample_filter     = [1,1],        # Resampling filter: [1,1] for DDPM++, [1,3,3,1] for NCSN++.
    ):
        assert embedding_type in ['fourier', 'positional']
        assert encoder_type in ['standard', 'skip', 'residual']
        assert decoder_type in ['standard', 'skip']

        super().__init__()
        self.label_dropout = label_dropout
        emb_channels = model_channels * channel_mult_emb
        noise_channels = model_channels * channel_mult_noise
        init = dict(init_mode='xavier_uniform')
        init_zero = dict(init_mode='xavier_uniform', init_weight=1e-5)
        init_attn = dict(init_mode='xavier_uniform', init_weight=np.sqrt(0.2))
        block_kwargs = dict(
            emb_channels=emb_channels, num_heads=1, dropout=dropout, skip_scale=np.sqrt(0.5), eps=1e-6,
            resample_filter=resample_filter, resample_proj=True, adaptive_scale=False,
            init=init, init_zero=init_zero, init_attn=init_attn,
        )

        # Mapping.
        self.map_noise = PositionalEmbedding(num_channels=noise_channels, endpoint=True) if embedding_type == 'positional' else FourierEmbedding(num_channels=noise_channels)
        self.map_label = Linear(in_features=label_dim, out_features=noise_channels, **init) if label_dim else None
        self.map_augment = Linear(in_features=augment_dim, out_features=noise_channels, bias=False, **init) if augment_dim else None
        self.map_layer0 = Linear(in_features=noise_channels, out_features=emb_channels, **init)
        self.map_layer1 = Linear(in_features=emb_channels, out_features=emb_channels, **init)

        # Encoder.
        self.enc = torch.nn.ModuleDict()
        cout = in_channels
        caux = in_channels
        # for level, mult in enumerate(channel_mult):
        res = img_resolution
        cin = cout
        cout = model_channels
        self.enc[f'{res}x{res}_conv'] = Conv2d(in_channels=cin, out_channels=cout, kernel=3, **init)
        for idx in range(num_blocks):
            cin = cout
            cout = model_channels # * channel_mult
            attn = (res in attn_resolutions)
            self.enc[f'{res}x{res}_block{idx}'] = UNetBlock(in_channels=cin, out_channels=cout, attention=attn, **block_kwargs)

        # Decoder.
        self.dec = torch.nn.ModuleDict()
        self.dec[f'{res}x{res}_aux_norm'] = GroupNorm(num_channels=cout, eps=1e-6)
        self.dec[f'{res}x{res}_aux_conv'] = Conv2d(in_channels=cout, out_channels=out_channels, kernel=3, **init_zero)

    def forward(self, x, noise_labels, cond, augment_labels=None):
        # Mapping.
        emb = self.map_noise(noise_labels)
        emb = emb.reshape(emb.shape[0], 2, -1).flip(1).reshape(*emb.shape) # swap sin/cos
        if self.map_label is not None:
            tmp = cond
            if self.training and self.label_dropout:
                tmp = tmp * (torch.rand([x.shape[0], 1], device=x.device) >= self.label_dropout).to(tmp.dtype)
            emb = emb + self.map_label(tmp * np.sqrt(self.map_label.in_features))
        if self.map_augment is not None and augment_labels is not None:
            emb = emb + self.map_augment(augment_labels)
        emb = silu(self.map_layer0(emb))
        emb = silu(self.map_layer1(emb))

        # Encoder.
        aux = x
        for name, block in self.enc.items():
            if 'aux_down' in name:
                aux = block(aux)
            elif 'aux_skip' in name:
                x = x + block(aux)
            elif 'aux_residual' in name:
                x = aux = (x + block(aux)) / np.sqrt(2)
            else:
                x = block(x, emb) if isinstance(block, UNetBlock) else block(x)

        # Decoder.
        aux = None
        tmp = None
        for name, block in self.dec.items():
            if 'aux_up' in name:
                aux = block(aux)
            elif 'aux_norm' in name:
                tmp = block(x)
            elif 'aux_conv' in name:
                tmp = block(silu(tmp))
                aux = tmp if aux is None else tmp + aux
            else:
                x = block(x, emb)
        return aux

In [ ]:
net = SongUNetResNet(img_resolution=32, in_channels=3, out_channels=3, 
               model_channels=128, channel_mult_emb=4, num_blocks=2, 
               attn_resolutions=[], 
               dropout=0.1, label_dropout=0, 
               embedding_type='positional', channel_mult_noise=1, 
               encoder_type='standard', decoder_type='standard', resample_filter=[1,1])